In [23]:
import threading
import numpy as np
import cv2
from PIL import Image, ImageTk
from tkinter import Tk, Frame, Button, BOTH, Label, Scale, Radiobutton       # Graphical User Inetrface Stuff
from tkinter import font as tkFont
import tkinter as tk

In [33]:
def masking(img):
    image_heigth = img.shape[0]
    image_width = img.shape[1]

    region_of_interest_vertices = [
        (0, image_heigth),
        (image_width/2, image_heigth/2),
        (image_width, image_heigth)
    ]

    mask = np.zeros_like(img)
    match_mask_color = 255
    cv2.fillPoly(mask, np.array([region_of_interest_vertices], np.int32), match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


In [34]:
def diffimg(t0, t1, t2):
    t0_masked = masking(t0)
    t1_masked = masking(t1)
    t2_masked = masking(t2)

    d1 = cv2.absdiff(t2_masked, t1_masked)
    d2 = cv2.absdiff(t1_masked, t0_masked)
    result = cv2.bitwise_and(d1, d2)
    ret, binary = cv2.threshold(result, 10, 255, cv2.THRESH_BINARY)
    return binary

In [36]:
input_video = cv2.VideoCapture("DashCam2.mp4")

t1 = cv2.cvtColor(input_video.read()[1], cv2.COLOR_RGB2GRAY)
t = cv2.cvtColor(input_video.read()[1], cv2.COLOR_RGB2GRAY)
t2 = cv2.cvtColor(input_video.read()[1], cv2.COLOR_RGB2GRAY)

while True:
    cv2.imshow("Diff", diffimg(t1, t, t2))
    t1 = t
    t = t2
    t2 = cv2.cvtColor(input_video.read()[1], cv2.COLOR_RGB2GRAY)

    key = cv2.waitKey(10)
    if key == 27:
        cv2.destroyAllWindows()
        break